In [62]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
from fairseq2.models.llama import LLaMABuilder,LLaMAConfig,LLaMATokenizer,LLaMAConfig,llama_archs
from fairseq2.data import VocabularyInfo
from transformers import AutoTokenizer,AutoConfig,AutoModelForCausalLM
import torch
import json
from pathlib import Path
from fairseq2.models.sequence import SequenceBatch
from fairseq2.nn.padding import PaddingMask
import torch.nn.functional as F
import random

In [ ]:
m = AutoModelForCausalLM.from_pretrained()
m.generate

In [2]:
with open(Path('/data/lz/models/CodeLlama-7b') / "params.json", "r") as f:
    params = json.loads(f.read())

In [3]:
ckpt = torch.load('/data/lz/models/CodeLlama-7b/consolidated.00.pth', map_location="cpu")

In [4]:
config = llama_archs.get_config('7b-code')

In [5]:
bdr = LLaMABuilder(config)

In [6]:
llama = bdr.build_model()

In [7]:
ckpt2 = {}
ckpt2['decoder_frontend.embed.weight'] = ckpt['tok_embeddings.weight']
ckpt2['decoder.layer_norm.weight'] = ckpt['norm.weight']
ckpt2['final_proj.weight'] = ckpt['output.weight']
for i in range(params['n_layers']):
    ckpt2[f'decoder.layers.{i}.self_attn_layer_norm.weight'] = ckpt[f'layers.{i}.attention_norm.weight']
    ckpt2[f'decoder.layers.{i}.self_attn.q_proj.weight'] = ckpt[f'layers.{i}.attention.wq.weight']
    ckpt2[f'decoder.layers.{i}.self_attn.k_proj.weight'] = ckpt[f'layers.{i}.attention.wk.weight']
    ckpt2[f'decoder.layers.{i}.self_attn.v_proj.weight'] = ckpt[f'layers.{i}.attention.wv.weight']
    ckpt2[f'decoder.layers.{i}.self_attn.output_proj.weight'] = ckpt[f'layers.{i}.attention.wo.weight']
    ckpt2[f'decoder.layers.{i}.ffn.gate_proj.weight'] = ckpt[f'layers.{i}.feed_forward.w1.weight']
    ckpt2[f'decoder.layers.{i}.ffn.output_proj.weight'] = ckpt[f'layers.{i}.feed_forward.w2.weight']
    ckpt2[f'decoder.layers.{i}.ffn.inner_proj.weight'] = ckpt[f'layers.{i}.feed_forward.w3.weight']
    ckpt2[f'decoder.layers.{i}.ffn_layer_norm.weight'] = ckpt[f'layers.{i}.ffn_norm.weight']

In [8]:
llama.load_state_dict(ckpt2)

<All keys matched successfully>

In [9]:
t = LLaMATokenizer('/data/lz/models/CodeLlama-7b/tokenizer.model')

In [72]:
torch.manual_seed(2618)
diversity_penalty_weight = 0.5
top_k = 5
top_p = 0.95
temperature = 0.8
s = 'def calculator('
seq = t.create_encoder()(s)[:-1]
for i in range(50):
    seqs = torch.stack([seq])
    mask = PaddingMask(torch.tensor([len(seqs)]),len(seqs))
    batch = SequenceBatch(seqs,mask)
    result = llama.forward(batch)
    logits = F.softmax(result.logits, dim=-1)
    last_token_logits = logits[0, -1, :]
    topk_values, topk_indices = torch.topk(last_token_logits,top_k) 
    probs = F.softmax(topk_values / temperature, dim=0)
    sorted_probs, sorted_indices = torch.sort(probs, descending=True) 
    cumulative_probs = torch.cumsum(sorted_probs, dim=0)
    selected_indices = sorted_indices[cumulative_probs <= top_p]
    next_token = topk_indices[random.choice(selected_indices)]
    
    
    if i>0 and last_token == next_token:
        last_token_logits[last_token] -= diversity_penalty_weight
        topk_values, topk_indices = torch.topk(last_token_logits,top_k)
        # topk_indices = torch.tensor(filter(lambda x:x>top_p,))
        next_token = random.choice(topk_indices)  
    seq = torch.concat([seq,torch.stack([next_token])])
    
    output = str(t.create_decoder()(seq)[0])
    print(output)
    last_token = next_token
    if last_token == 2:
        print('</s>')
        break
    


def calculator(($
def calculator(($start
def calculator(($start)
def calculator(($start) +
def calculator(($start) + $
def calculator(($start) + $start
def calculator(($start) + $start)
def calculator(($start) + $start) 
def calculator(($start) + $start) 

def calculator(($start) + $start) 
(
def calculator(($start) + $start) 
(

def calculator(($start) + $start) 
(
\
def calculator(($start) + $start) 
(
\n
def calculator(($start) + $start) 
(
\n calcul
def calculator(($start) + $start) 
(
\n calculates
def calculator(($start) + $start) 
(
\n calculates 
def calculator(($start) + $start) 
(
\n calculates 2
def calculator(($start) + $start) 
(
\n calculates 2 
def calculator(($start) + $start) 
(
\n calculates 2 2
def calculator(($start) + $start) 
(
\n calculates 2 2 
def calculator(($start) + $start) 
(
\n calculates 2 2 3
def calculator(($start) + $start) 
(
\n calculates 2 2 3

def calculator(($start) + $start) 
(
\n calculates 2 2 3
\
def calculator(($start) + $start) 
(
\n calcula